In [1]:
import pandas as pd 

es_df = pd.read_csv('ES_meta.csv')
es_df



,filename,fold,target,category
0,./data/wav_정상품_V1a/#01_SX2_Normal_ES_RH_1/#01_...,1,0,Normal
1,./data/wav_정상품_V1a/#01_SX2_Normal_ES_RH_2/#01_...,1,0,Normal
2,./data/wav_정상품_V1a/#01_SX2_Normal_ES_LH_1/#01_...,1,0,Normal
3,./data/wav_정상품_V1a/#01_SX2_Normal_ES_LH_2/#01_...,1,0,Normal
4,./data/wav_정상품_V1a/#02_SX2_Normal_ES_LH_1/#02_...,1,0,Normal
...,...,...,...,...
540,./data/wav_문제품DATAa/#44_SX2_C_ES_LH_1/#44_SX2_...,1,3,SG
541,./data/wav_문제품DATAa/#33_SX2_B_ES_RH_1/#33_SX2_...,1,3,SG
542,./data/wav_문제품DATAa/#33_SX2_B_ES_RH_2/#33_SX2_...,1,3,SG
543,./data/wav_문제품DATAa/#21_SX2_A_ES_RH_1/#21_SX2_...,1,3,SG


In [2]:
line_df = pd.read_csv('LINE_meta.csv')
line_df

,filename,fold,target,category
0,./data/wav_정상품_V1a/#01_SX2_Normal_Line_RH_2/#0...,1,0,Normal
1,./data/wav_정상품_V1a/#01_SX2_Normal_Line_RH_1/#0...,1,0,Normal
2,./data/wav_정상품_V1a/#01_SX2_Normal_Line_LH_2/#0...,1,0,Normal
3,./data/wav_정상품_V1a/#01_SX2_Normal_Line_LH_1/#0...,1,0,Normal
4,./data/wav_정상품_V1a/#02_SX2_Normal_Line_RH_2/#0...,1,0,Normal
...,...,...,...,...
545,./data/wav_문제품DATAa/#32_SX2_A_LINE_LH_1/#32_SX...,1,3,SG
546,./data/wav_문제품DATAa/#33_SX2_B_LINE_RH_2/#33_SX...,1,3,SG
547,./data/wav_문제품DATAa/#33_SX2_B_LINE_RH_1/#33_SX...,1,3,SG
548,./data/wav_문제품DATAa/#21_SX2_A_LINE_RH_2/#21_SX...,1,3,SG


In [3]:
import librosa
import numpy as np
import torch
# .wav 파일 불러오기


# 예제 사용

def get_tensor(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # sr=None을 설정하여 원본 샘플링 레이트를 유지

    # NumPy 배열을 PyTorch 텐서로 변환
    tensor_data = torch.tensor(audio)

    # 결과 확인
    def perform_stft(waveform, n_fft=512, win_length=None, hop_length=None, window_fn=torch.hann_window):
        stft = torch.stft(waveform, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window=window_fn(n_fft), return_complex=True)
        return stft

    stft_result = perform_stft(tensor_data)

    def apply_a_weighting_librosa(stft_result, sample_rate):
        # STFT 결과로부터 FFT 크기 추정
        n_fft = (stft_result.size(0) - 1) * 2
        
        # 주파수 bins 계산
        freqs = np.linspace(0, sample_rate / 2, stft_result.size(0))
        
        # A-weighting dB 값 계산
        a_weighting_db = librosa.A_weighting(freqs)
        
        # dB를 power scale로 변환
        a_weighting_scale = librosa.db_to_amplitude(a_weighting_db)
        
        # 텐서로 변환
        a_weighting_scale_tensor = torch.from_numpy(a_weighting_scale).to(stft_result.dtype)
        
        # A-weighted STFT 계산
        a_weighted_stft = torch.abs(stft_result).log10() + a_weighting_scale_tensor.unsqueeze(1)  # 주파수 차원에 적용
        #a_weighted_stft = torch.abs(stft_result) * a_weighting_scale_tensor.unsqueeze(1)  # 주파수 차원에 적용

        return a_weighted_stft

    sample_rate = 12800  # 샘플 레이트 예제 값
    # A-weighting 적용
    a_weighted_result = apply_a_weighting_librosa(stft_result, sample_rate)
    _3k_result = a_weighted_result[32:129,:] # 하나당 25이므로, 800 하려면
    return torch.abs(_3k_result) 


In [4]:
es_grr_list = []
es_sg_list = []
es_dol_list = []


for idx in es_df.index:
    row = es_df.loc[idx]
    filename = row['filename']
    if row['category'] == 'GRR':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            es_grr_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'SG':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            es_sg_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'DOL':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            es_dol_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)
    


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [5]:
es_grr_list[0].shape

torch.Size([1, 97, 1000])

In [6]:
rps_df = pd.read_csv('1.0RPS_meta.csv')

rps_grr_list = []
rps_sg_list = []
rps_dol_list = []

for idx in rps_df.index:
    row = rps_df.loc[idx]
    filename= row['filename']
    if row['category'] == 'GRR':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            rps_grr_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'SG':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            rps_sg_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'DOL':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            rps_dol_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__() # (97, 1000) 
        # Encoder
        self.encoder = nn.Sequential( 
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 49, 500)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 25, 250)
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 64, 13, 125)
            nn.ReLU(True),
            nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 7, 63)
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 4, 32)
            nn.ReLU(True),
            nn.Conv2d(16, 1, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 1, 2, 16)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output shape: (b, 16, 4, 32)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2, padding=(1, 1), output_padding=(0, 0)),  # Output shape: (b, 32, 7, 63) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2, padding=(1,1), output_padding=(0 ,0)),  # Output shape: (b, 64, 13, 125) 
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=(1,1), output_padding=(0, 1)),  # Output shape: (b, 32, 25, 250)  
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 16, 49, 500)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 1, 97, 1000) 
        )
# output padding must be smaller than stride
    def forward(self, x):
        mid = self.encoder(x)
        x = self.decoder(mid)
        return x
    
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


test_autoencoder = Autoencoder()

input_tensor = torch.randn(1, 1, 97, 1000)  # (batch_size, channels, height, width)

# Forward pass through the autoencoder
output_tensor = test_autoencoder(input_tensor)

# Print the shapes of input and output tensors
print("Input Tensor Shape:", input_tensor.shape)
print("Output Tensor Shape:", output_tensor.shape)



Input Tensor Shape: torch.Size([1, 1, 97, 1000])
Output Tensor Shape: torch.Size([1, 1, 97, 1000])


In [8]:
def train(dataset_list):
    dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    customdataset = CustomDataset(dataset_list)
    train_loader = DataLoader(customdataset, batch_size=32, shuffle=True)


    # Initialize the model, loss function, and optimizer
    autoencoder = Autoencoder().to(dev)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 100
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader): # i-th batch
            inputs = data
            inputs = inputs.to(dev)

            optimizer.zero_grad()

            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'epoch : {epoch}, loss: {running_loss}')


    print('Finished Training')

    encoded_list = []
    with torch.no_grad():
        for i in dataset_list:
            encoded_list.append(autoencoder.encoder(i.to(dev)).flatten())

    return encoded_list
           


In [9]:
encoded_es_grr_list = train(es_grr_list)
encoded_es_dol_list = train(es_dol_list)
encoded_es_sg_list = train(es_sg_list)

encoded_rps_grr_list = train(rps_grr_list)
encoded_rps_dol_list = train(rps_dol_list)
encoded_rps_sg_list = train(rps_sg_list)


epoch : 0, loss: 1.6104599833488464
epoch : 1, loss: 1.6633005738258362
epoch : 2, loss: 1.4890207648277283
epoch : 3, loss: 1.6116323471069336
epoch : 4, loss: 1.612727165222168
epoch : 5, loss: 1.5507891178131104
epoch : 6, loss: 1.4642230868339539
epoch : 7, loss: 1.531641960144043
epoch : 8, loss: 1.5158032774925232
epoch : 9, loss: 1.4795319437980652
epoch : 10, loss: 1.3708306550979614
epoch : 11, loss: 1.3866477012634277
epoch : 12, loss: 1.3150774240493774
epoch : 13, loss: 1.235659122467041
epoch : 14, loss: 1.2373822927474976
epoch : 15, loss: 1.0702800154685974
epoch : 16, loss: 1.0040754973888397
epoch : 17, loss: 0.9827935695648193
epoch : 18, loss: 0.9575589001178741
epoch : 19, loss: 0.9589625895023346
epoch : 20, loss: 0.8616147637367249
epoch : 21, loss: 0.8274655342102051
epoch : 22, loss: 0.7354930937290192
epoch : 23, loss: 0.7025419473648071
epoch : 24, loss: 0.6476605832576752
epoch : 25, loss: 0.6100171506404877
epoch : 26, loss: 0.552794486284256
epoch : 27, los

In [10]:
idx = 16
print(encoded_es_grr_list[idx])
print(encoded_es_sg_list[idx])

tensor([-24.9774, -31.3812, -26.6797, -24.6731, -21.9418, -20.0058, -19.4663,
        -17.5291, -15.9381, -14.8090, -13.5477, -12.1601, -11.3389,  -9.2548,
         -7.9598,  -6.4812, -31.1909, -35.4601, -31.4720, -28.8491, -23.3081,
        -21.2620, -17.9917, -14.3676, -11.6996, -10.2890,  -9.6853,  -8.3825,
         -6.4104,  -4.2516,  -3.9875,  -2.5728], device='cuda:0')
tensor([-40.4090, -42.7548, -37.5884, -35.8237, -32.7759, -31.5517, -30.7348,
        -29.1994, -27.5940, -25.1336, -23.3887, -19.8454, -18.1463, -17.0682,
        -14.2170, -10.4736, -44.7814, -49.0910, -43.5504, -39.1361, -35.2983,
        -32.1538, -28.2776, -24.5757, -21.2351, -18.6618, -18.6314, -11.7865,
        -15.0560, -13.4057,  -7.2478,  -7.9574], device='cuda:0')


In [37]:
from sklearn.cluster import KMeans

es_concated = encoded_es_grr_list +  encoded_es_sg_list +  encoded_es_dol_list
es_concated = torch.stack(es_concated).cpu().numpy()

# Run k-means clustering
kmeans = KMeans(n_clusters=3, random_state=0)
es_cluster_labels = kmeans.fit_predict(es_concated)




/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [51]:
print(len(encoded_es_grr_list))
print(len(encoded_es_sg_list))
print(len(encoded_es_dol_list))

38
98
107


In [53]:
es_clstr_grr_cnt={0:0, 1:0, 2:0} 
es_clstr_sg_cnt={0:0, 1:0, 2:0} 
es_clstr_dol_cnt={0:0, 1:0, 2:0} 

for i, cl_label in enumerate(es_cluster_labels):
    if i < len(encoded_es_grr_list):
        es_clstr_grr_cnt[cl_label]+= 1
    elif i < len(encoded_es_grr_list) + len(encoded_es_sg_list):
        es_clstr_sg_cnt[cl_label]+= 1
    else:
        es_clstr_dol_cnt[cl_label]+= 1

      
print(es_clstr_grr_cnt)
print(es_clstr_sg_cnt)
print(es_clstr_dol_cnt) # grrsg, dol 의 필요성. 모비스에선 grrdol, sg라고 했는데. 



{0: 36, 1: 0, 2: 2}
{0: 27, 1: 0, 2: 71}
{0: 0, 1: 107, 2: 0}


In [54]:
es_clstr_grrdol_cnt={0:0, 1:0} 
es_clstr_ssg_cnt={0:0, 1:0} 

_2kmeans = KMeans(n_clusters=2, random_state=0)
_2es_cluster_labels = _2kmeans.fit_predict(es_concated)

for i, cl_label in enumerate(_2es_cluster_labels):
    if i < len(encoded_es_grr_list):
        es_clstr_grrdol_cnt[cl_label]+= 1
    elif i < len(encoded_es_grr_list) + len(encoded_es_sg_list) :
        es_clstr_ssg_cnt[cl_label] += 1
    else:
        es_clstr_grrdol_cnt[cl_label] +=1

print(es_clstr_grrdol_cnt)
print(es_clstr_ssg_cnt)


{0: 38, 1: 107}
{0: 98, 1: 0}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [57]:
es_clstr_grrsg_cnt={0:0, 1:0} 
es_clstr_dol_cnt={0:0, 1:0} 

_2es_cluster_labels = _2kmeans.fit_predict(es_concated)

for i, cl_label in enumerate(_2es_cluster_labels):
    if i < len(encoded_es_grr_list):
        es_clstr_grrsg_cnt[cl_label]+= 1
    elif i < len(encoded_es_grr_list) + len(encoded_es_sg_list) :
        es_clstr_grrsg_cnt[cl_label] += 1
    else:
        es_clstr_dol_cnt[cl_label] +=1

print(es_clstr_grrsg_cnt)
print(es_clstr_dol_cnt) # grrsg | dol 이 잘된다. 


{0: 136, 1: 0}
{0: 0, 1: 107}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [63]:
print(len(encoded_rps_grr_list))
print(len(encoded_rps_sg_list))
print(len(encoded_rps_dol_list))

13
46
43


In [61]:
rps_concated = encoded_rps_grr_list +  encoded_rps_sg_list +  encoded_rps_dol_list
rps_concated = torch.stack(rps_concated).cpu().numpy()

# Run k-means clustering
rps_cluster_labels = _2kmeans.fit_predict(rps_concated)

rps_clstr_grrsg_cnt={0:0, 1:0} 
rps_clstr_dol_cnt={0:0, 1:0} 

for i, cl_label in enumerate(rps_cluster_labels):
    if i < len(encoded_rps_grr_list):
        rps_clstr_grrsg_cnt[cl_label]+= 1
    elif i < len(encoded_rps_grr_list) + len(encoded_rps_sg_list) :
        rps_clstr_grrsg_cnt[cl_label] += 1
    else:
        rps_clstr_dol_cnt[cl_label] +=1

print(rps_clstr_grrsg_cnt)
print(rps_clstr_dol_cnt)




{0: 49, 1: 10}
{0: 0, 1: 43}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [65]:
# Run k-means clustering
rps_cluster_labels = _2kmeans.fit_predict(rps_concated)

rps_clstr_grrdol_cnt={0:0, 1:0} 
rps_clstr_sg_cnt={0:0, 1:0} 

for i, cl_label in enumerate(rps_cluster_labels):
    if i < len(encoded_rps_grr_list):
        rps_clstr_grrdol_cnt[cl_label]+= 1
    elif i < len(encoded_rps_grr_list) + len(encoded_rps_sg_list) :
        rps_clstr_sg_cnt[cl_label] += 1
    else:
        rps_clstr_grrdol_cnt[cl_label] +=1

print(rps_clstr_grrdol_cnt)
print(rps_clstr_sg_cnt)   # 여기선 또 grrdol | sg 이 잘된다. 




{0: 3, 1: 53}
{0: 46, 1: 0}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [64]:
rps_concated = encoded_rps_grr_list +  encoded_rps_sg_list +  encoded_rps_dol_list
rps_concated = torch.stack(rps_concated).cpu().numpy()

# Run k-means clustering
rps_cluster_labels = kmeans.fit_predict(rps_concated)

rps_clstr_grr_cnt={0:0, 1:0, 2:0} # total should be: 298
rps_clstr_sg_cnt={0:0, 1:0, 2:0} # total should be: 298
rps_clstr_dol_cnt={0:0, 1:0, 2:0} # total should be: 298

for i, cl_label in enumerate(rps_cluster_labels):
    if i < len(encoded_rps_grr_list):
        rps_clstr_grr_cnt[cl_label]+= 1
    elif i < len(encoded_rps_grr_list) + len(encoded_rps_sg_list) :
        rps_clstr_sg_cnt[cl_label] += 1
    else:
        rps_clstr_dol_cnt[cl_label] +=1

print(rps_clstr_grr_cnt)
print(rps_clstr_sg_cnt)
print(rps_clstr_dol_cnt)




{0: 13, 1: 0, 2: 0}
{0: 0, 1: 46, 2: 0}
{0: 20, 1: 0, 2: 23}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [20]:
len(encoded_rps_sg_list)

46

In [66]:
line_df = pd.read_csv('LINE_meta.csv')

line_grr_list = []
line_sg_list = []
line_dol_list = []

for idx in line_df.index:
    row = line_df.loc[idx]
    filename= row['filename']
    if row['category'] == 'GRR':
        ten = get_tensor(filename)
        if ten.shape[1]>=400:
            line_grr_list.append(ten[:,:400].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'SG':
        ten = get_tensor(filename)
        if ten.shape[1]>=400: # 400으로?
            line_sg_list.append(ten[:,:400].unsqueeze(0))
        else:
            print(ten.shape)

    elif row['category'] == 'DOL':
        ten = get_tensor(filename)
        if ten.shape[1]>=400:
            line_dol_list.append(ten[:,:400].unsqueeze(0))
        else:
            print(ten.shape)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


torch.Size([97, 352])
torch.Size([97, 355])
torch.Size([97, 357])
torch.Size([97, 368])


In [67]:
line_grr_list[0].shape

torch.Size([1, 97, 400])

In [68]:
class line_Autoencoder(nn.Module):
    def __init__(self):
        super(line_Autoencoder, self).__init__() # (97, 400) 이라고 했을 때 
        # Encoder
        self.encoder = nn.Sequential( # Given groups=1, weight of size [16, 128, 3, 3], expected input[1, 32, 8, 95] to have 128 channels, but got 32 channels instead
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 49, 200)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 25, 100)
            nn.ReLU(True),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 64, 13, 50)
            nn.ReLU(True),
            nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 7, 25)
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 4, 13)
            
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 16, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,0)),  # Output shape: (b, 16, 7, 25)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2, padding=(1, 1), output_padding=(0,1)),  # Output shape: (b, 32, 13, 50) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 64, 25, 100) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 32, 49, 200)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 16, 97, 400)  
            
        )
# output padding must be smaller than stride
    def forward(self, x):
        mid = self.encoder(x)
        x = self.decoder(mid)
        return x

In [69]:
def line_train(dataset_list):
    dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    customdataset = CustomDataset(dataset_list)
    train_loader = DataLoader(customdataset, batch_size=32, shuffle=True)


    # Initialize the model, loss function, and optimizer
    autoencoder = line_Autoencoder().to(dev)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 100
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader): # i-th batch
            inputs = data
            inputs = inputs.to(dev)

            optimizer.zero_grad()

            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'epoch : {epoch}, loss: {running_loss}')


    print('Finished Training')

    encoded_list = []
    with torch.no_grad():
        for i in dataset_list:
            encoded_list.append(autoencoder.encoder(i.to(dev)).flatten())

    return encoded_list
           


In [70]:
encoded_line_grr_list = line_train(line_grr_list)
encoded_line_sg_list = line_train(line_sg_list)
encoded_line_dol_list = line_train(line_dol_list)


line_concated = encoded_line_grr_list +  encoded_line_sg_list +  encoded_line_dol_list
line_concated = torch.stack(line_concated).cpu().numpy()

# Run k-means clustering
line_cluster_labels = kmeans.fit_predict(line_concated)




epoch : 0, loss: 1.762221336364746
epoch : 1, loss: 1.7548028230667114
epoch : 2, loss: 1.7473225593566895
epoch : 3, loss: 1.7397414445877075
epoch : 4, loss: 1.732003927230835
epoch : 5, loss: 1.7241140604019165
epoch : 6, loss: 1.716019868850708
epoch : 7, loss: 1.7076514959335327
epoch : 8, loss: 1.6989134550094604
epoch : 9, loss: 1.6896555423736572
epoch : 10, loss: 1.6796834468841553
epoch : 11, loss: 1.6690305471420288
epoch : 12, loss: 1.656985878944397
epoch : 13, loss: 1.6423554420471191
epoch : 14, loss: 1.6235017776489258
epoch : 15, loss: 1.5980122089385986
epoch : 16, loss: 1.562334418296814
epoch : 17, loss: 1.511401653289795
epoch : 18, loss: 1.4379178285598755
epoch : 19, loss: 1.3332713842391968
epoch : 20, loss: 1.1909265518188477
epoch : 21, loss: 1.0160973072052002
epoch : 22, loss: 0.855827808380127
epoch : 23, loss: 0.8524380922317505
epoch : 24, loss: 0.9180907607078552
epoch : 25, loss: 0.8029083013534546
epoch : 26, loss: 0.6627845168113708
epoch : 27, loss: 

/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [75]:
line_concated = encoded_line_grr_list +  encoded_line_sg_list +  encoded_line_dol_list
line_concated = torch.stack(line_concated).cpu().numpy()

# Run k-means clustering
line_cluster_labels = kmeans.fit_predict(line_concated)

line_clstr_grr_cnt={0:0, 1:0, 2:0} # total should be: 298
line_clstr_sg_cnt={0:0, 1:0, 2:0} # total should be: 298
line_clstr_dol_cnt={0:0, 1:0, 2:0} # total should be: 298

for i, cl_label in enumerate(line_cluster_labels):
    if i < len(encoded_line_grr_list):
        line_clstr_grr_cnt[cl_label]+= 1
    elif i < len(encoded_line_grr_list) + len(encoded_line_sg_list) :
        line_clstr_sg_cnt[cl_label] += 1
    else:
        line_clstr_dol_cnt[cl_label] +=1

print(line_clstr_grr_cnt)
print(line_clstr_sg_cnt)
print(line_clstr_dol_cnt)  # line에서는, grrsg | dol 




/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{0: 23, 1: 0, 2: 1}
{0: 12, 1: 0, 2: 94}
{0: 0, 1: 102, 2: 0}


In [76]:
print(len(encoded_line_grr_list))
print(len(encoded_line_sg_list))
print(len(encoded_line_dol_list))

24
106
102


In [78]:
line_clstr_grrsg_cnt={0:0, 1:0} 
line_clstr_dol_cnt={0:0, 1:0} 

_2line_cluster_labels = _2kmeans.fit_predict(line_concated)

for i, cl_label in enumerate(_2line_cluster_labels):
    if i < len(encoded_line_grr_list):
        line_clstr_grrsg_cnt[cl_label]+= 1
    elif i < len(encoded_line_grr_list) + len(encoded_line_sg_list) :
        line_clstr_grrsg_cnt[cl_label] += 1
    else:
        line_clstr_dol_cnt[cl_label] +=1

print(line_clstr_grrsg_cnt)
print(line_clstr_dol_cnt) # grrsg | dol 이 잘된다. 


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{0: 130, 1: 0}
{0: 0, 1: 102}


In [79]:
line_clstr_grrdol_cnt={0:0, 1:0} 
line_clstr_sg_cnt={0:0, 1:0} 

_2line_cluster_labels = _2kmeans.fit_predict(line_concated)

for i, cl_label in enumerate(_2line_cluster_labels):
    if i < len(encoded_line_grr_list):
        line_clstr_grrdol_cnt[cl_label]+= 1
    elif i < len(encoded_line_grr_list) + len(encoded_line_sg_list) :
        line_clstr_sg_cnt[cl_label] += 1
    else:
        line_clstr_grrdol_cnt[cl_label] +=1

print(line_clstr_grrdol_cnt)
print(line_clstr_sg_cnt) 


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{0: 24, 1: 102}
{0: 106, 1: 0}


결론: rps만 빼면 모두 grrsg | dol 이 가장 좋았다.